# Part A: Data Understanding & Quality Checks
## K&Co Cloud Cost Intelligence Platform

This notebook performs comprehensive data profiling on AWS and GCP billing data to:
- Understand data structure and volume
- Identify data quality issues
- Document risks and remediation strategies

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Datasets

In [ ]:
# Load AWS billing data
aws_df = pd.read_csv('../data/aws_line_items_12mo.csv')
print("AWS Data Loaded Successfully")
print(f"Shape: {aws_df.shape}")
print(f"\nFirst few rows:")
aws_df.head()

In [ ]:
# Load GCP billing data
gcp_df = pd.read_csv('../data/gcp_billing_12mo.csv')
print("GCP Data Loaded Successfully")
print(f"Shape: {gcp_df.shape}")
print(f"\nFirst few rows:")
gcp_df.head()

## 2. Basic Data Profiling
### 2.1 Row Counts and Structure

In [ ]:
# Create profiling summary
profiling_summary = pd.DataFrame({
    'Dataset': ['AWS', 'GCP'],
    'Row Count': [len(aws_df), len(gcp_df)],
    'Column Count': [len(aws_df.columns), len(gcp_df.columns)],
    'Memory Usage (MB)': [
        aws_df.memory_usage(deep=True).sum() / 1024**2,
        gcp_df.memory_usage(deep=True).sum() / 1024**2
    ]
})

print("=" * 60)
print("DATA PROFILING SUMMARY")
print("=" * 60)
print(profiling_summary.to_string(index=False))
print("\n")

In [ ]:
# Column information
print("AWS Columns:")
print(aws_df.dtypes)
print("\n" + "="*60 + "\n")
print("GCP Columns:")
print(gcp_df.dtypes)

### 2.2 Missing/Null Values Analysis

In [ ]:
# Check for missing values in AWS data
aws_missing = pd.DataFrame({
    'Column': aws_df.columns,
    'Missing Count': aws_df.isnull().sum(),
    'Missing %': (aws_df.isnull().sum() / len(aws_df) * 100).round(2)
})

print("AWS Missing Values:")
print(aws_missing.to_string(index=False))
print("\n")

In [ ]:
# Check for missing values in GCP data
gcp_missing = pd.DataFrame({
    'Column': gcp_df.columns,
    'Missing Count': gcp_df.isnull().sum(),
    'Missing %': (gcp_df.isnull().sum() / len(gcp_df) * 100).round(2)
})

print("GCP Missing Values:")
print(gcp_missing.to_string(index=False))

### 2.3 Duplicate Records Check

In [ ]:
# Check for duplicate rows
aws_duplicates = aws_df.duplicated().sum()
gcp_duplicates = gcp_df.duplicated().sum()

print(f"AWS Duplicate Rows: {aws_duplicates} ({aws_duplicates/len(aws_df)*100:.2f}%)")
print(f"GCP Duplicate Rows: {gcp_duplicates} ({gcp_duplicates/len(gcp_df)*100:.2f}%)")

# Check for duplicates on key columns (date, service, team, env)
aws_key_duplicates = aws_df.duplicated(subset=['date', 'account_id', 'service', 'team', 'env']).sum()
gcp_key_duplicates = gcp_df.duplicated(subset=['date', 'project_id', 'service', 'team', 'env']).sum()

print(f"\nAWS Duplicates on Key Columns: {aws_key_duplicates}")
print(f"GCP Duplicates on Key Columns: {gcp_key_duplicates}")

### 2.4 Date Range Analysis

In [ ]:
# Convert date columns to datetime
aws_df['date'] = pd.to_datetime(aws_df['date'])
gcp_df['date'] = pd.to_datetime(gcp_df['date'])

# Analyze date ranges
print("AWS Date Range:")
print(f"  Min Date: {aws_df['date'].min()}")
print(f"  Max Date: {aws_df['date'].max()}")
print(f"  Date Span: {(aws_df['date'].max() - aws_df['date'].min()).days} days")
print(f"  Unique Dates: {aws_df['date'].nunique()}")

print("\nGCP Date Range:")
print(f"  Min Date: {gcp_df['date'].min()}")
print(f"  Max Date: {gcp_df['date'].max()}")
print(f"  Date Span: {(gcp_df['date'].max() - gcp_df['date'].min()).days} days")
print(f"  Unique Dates: {gcp_df['date'].nunique()}")

In [ ]:
# Check for unexpected date gaps
aws_date_range = pd.date_range(start=aws_df['date'].min(), end=aws_df['date'].max(), freq='D')
aws_missing_dates = set(aws_date_range) - set(aws_df['date'].unique())

gcp_date_range = pd.date_range(start=gcp_df['date'].min(), end=gcp_df['date'].max(), freq='D')
gcp_missing_dates = set(gcp_date_range) - set(gcp_df['date'].unique())

print(f"AWS Missing Dates: {len(aws_missing_dates)}")
if len(aws_missing_dates) > 0 and len(aws_missing_dates) < 10:
    print(f"  {sorted(aws_missing_dates)}")

print(f"\nGCP Missing Dates: {len(gcp_missing_dates)}")
if len(gcp_missing_dates) > 0 and len(gcp_missing_dates) < 10:
    print(f"  {sorted(gcp_missing_dates)}")

### 2.5 Environment Values Analysis

In [ ]:
# Analyze environment values
print("AWS Environment Values:")
print(aws_df['env'].value_counts())
print(f"\nUnique Environments: {aws_df['env'].unique()}")

print("\n" + "="*60 + "\n")

print("GCP Environment Values:")
print(gcp_df['env'].value_counts())
print(f"\nUnique Environments: {gcp_df['env'].unique()}")

### 2.6 Service Names Analysis

In [ ]:
# Analyze service distribution
print("AWS Services:")
print(aws_df['service'].value_counts())
print(f"\nUnique Services: {aws_df['service'].nunique()}")

print("\n" + "="*60 + "\n")

print("GCP Services:")
print(gcp_df['service'].value_counts())
print(f"\nUnique Services: {gcp_df['service'].nunique()}")

### 2.7 Team Analysis

In [ ]:
# Analyze team distribution
print("AWS Teams:")
print(aws_df['team'].value_counts())

print("\n" + "="*60 + "\n")

print("GCP Teams:")
print(gcp_df['team'].value_counts())

### 2.8 Cost Analysis

In [ ]:
# Statistical summary of costs
print("AWS Cost Statistics:")
print(aws_df['cost_usd'].describe())
print(f"\nNegative Costs: {(aws_df['cost_usd'] < 0).sum()} records")
print(f"Zero Costs: {(aws_df['cost_usd'] == 0).sum()} records")

print("\n" + "="*60 + "\n")

print("GCP Cost Statistics:")
print(gcp_df['cost_usd'].describe())
print(f"\nNegative Costs: {(gcp_df['cost_usd'] < 0).sum()} records")
print(f"Zero Costs: {(gcp_df['cost_usd'] == 0).sum()} records")

In [ ]:
# Visualize cost distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# AWS cost distribution
axes[0].hist(aws_df['cost_usd'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('AWS Cost Distribution')
axes[0].set_xlabel('Cost (USD)')
axes[0].set_ylabel('Frequency')
axes[0].axvline(aws_df['cost_usd'].mean(), color='red', linestyle='--', label=f"Mean: ${aws_df['cost_usd'].mean():.2f}")
axes[0].legend()

# GCP cost distribution
axes[1].hist(gcp_df['cost_usd'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1].set_title('GCP Cost Distribution')
axes[1].set_xlabel('Cost (USD)')
axes[1].set_ylabel('Frequency')
axes[1].axvline(gcp_df['cost_usd'].mean(), color='red', linestyle='--', label=f"Mean: ${gcp_df['cost_usd'].mean():.2f}")
axes[1].legend()

plt.tight_layout()
plt.show()

### 2.9 Account/Project ID Analysis

In [ ]:
# Analyze account/project IDs
print("AWS Account IDs:")
print(aws_df['account_id'].value_counts())
print(f"\nUnique Account IDs: {aws_df['account_id'].nunique()}")

print("\n" + "="*60 + "\n")

print("GCP Project IDs:")
print(gcp_df['project_id'].value_counts())
print(f"\nUnique Project IDs: {gcp_df['project_id'].nunique()}")

## 3. Data Quality Risks

Based on the profiling above, here are the identified data quality risks:

In [ ]:
# Summary of data quality findings
quality_risks = [
    {
        'Risk': 'Negative Cost Values',
        'Description': f'Found {(aws_df["cost_usd"] < 0).sum()} negative costs in AWS and {(gcp_df["cost_usd"] < 0).sum()} in GCP',
        'Impact': 'Can skew financial reporting and analytics',
        'Remediation': 'Investigate if these are credits/refunds. Create separate column for credits or flag them explicitly.'
    },
    {
        'Risk': 'Duplicate Records on Key Columns',
        'Description': f'Found {aws_key_duplicates} duplicates in AWS and {gcp_key_duplicates} in GCP on date+account/project+service+team+env',
        'Impact': 'Double-counting costs leading to inflated spend reports',
        'Remediation': 'Implement deduplication logic based on composite key. Aggregate costs if legitimate multiple entries.'
    },
    {
        'Risk': 'Inconsistent Service Naming',
        'Description': 'Same services appear in both AWS and GCP (EC2, RDS, S3, Lambda, EKS) - likely synthetic data',
        'Impact': 'Confusion in cross-cloud analysis; GCP should use different service names',
        'Remediation': 'Create service mapping table to standardize names across clouds. Use cloud_provider prefix.'
    },
    {
        'Risk': 'Missing Date Continuity',
        'Description': f'Potential gaps in daily data: {len(aws_missing_dates)} missing dates in AWS, {len(gcp_missing_dates)} in GCP',
        'Impact': 'Incomplete time-series analysis and trending',
        'Remediation': 'Implement data completeness checks. Fill gaps with zero-cost records or flag missing days.'
    },
    {
        'Risk': 'No Data Type Validation',
        'Description': 'Columns loaded as generic types; no explicit validation of account_id, project_id formats',
        'Impact': 'Invalid IDs could enter system undetected',
        'Remediation': 'Implement schema validation with expected data types and regex patterns for IDs.'
    },
    {
        'Risk': 'Wide Cost Range Without Outlier Detection',
        'Description': f'AWS costs range from ${aws_df["cost_usd"].min():.2f} to ${aws_df["cost_usd"].max():.2f}',
        'Impact': 'Anomalous spikes may go unnoticed without automated detection',
        'Remediation': 'Implement statistical outlier detection (IQR, Z-score) and alerting for cost anomalies.'
    },
    {
        'Risk': 'No Referential Integrity Checks',
        'Description': 'Team, service, env values not validated against master lists',
        'Impact': 'Typos and invalid values can fragment reporting',
        'Remediation': 'Create dimension tables with valid values. Enforce foreign key constraints in warehouse.'
    }
]

# Display risks in a formatted table
risks_df = pd.DataFrame(quality_risks)
print("\n" + "="*80)
print("DATA QUALITY RISKS IDENTIFIED")
print("="*80 + "\n")

for idx, risk in enumerate(risks_df.to_dict('records'), 1):
    print(f"Risk #{idx}: {risk['Risk']}")
    print(f"  Description: {risk['Description']}")
    print(f"  Impact: {risk['Impact']}")
    print(f"  Remediation: {risk['Remediation']}")
    print()

## 4. Summary Statistics Table

In [ ]:
# Create comprehensive summary table
summary_stats = pd.DataFrame({
    'Metric': [
        'Total Records',
        'Date Range',
        'Unique Dates',
        'Missing Dates',
        'Unique Services',
        'Unique Teams',
        'Unique Environments',
        'Unique Accounts/Projects',
        'Total Cost (USD)',
        'Average Daily Cost (USD)',
        'Negative Cost Records',
        'Duplicate Records',
        'Missing Values'
    ],
    'AWS': [
        f"{len(aws_df):,}",
        f"{aws_df['date'].min().date()} to {aws_df['date'].max().date()}",
        f"{aws_df['date'].nunique()}",
        f"{len(aws_missing_dates)}",
        f"{aws_df['service'].nunique()}",
        f"{aws_df['team'].nunique()}",
        f"{aws_df['env'].nunique()}",
        f"{aws_df['account_id'].nunique()}",
        f"${aws_df['cost_usd'].sum():,.2f}",
        f"${aws_df.groupby('date')['cost_usd'].sum().mean():,.2f}",
        f"{(aws_df['cost_usd'] < 0).sum()}",
        f"{aws_key_duplicates}",
        f"{aws_df.isnull().sum().sum()}"
    ],
    'GCP': [
        f"{len(gcp_df):,}",
        f"{gcp_df['date'].min().date()} to {gcp_df['date'].max().date()}",
        f"{gcp_df['date'].nunique()}",
        f"{len(gcp_missing_dates)}",
        f"{gcp_df['service'].nunique()}",
        f"{gcp_df['team'].nunique()}",
        f"{gcp_df['env'].nunique()}",
        f"{gcp_df['project_id'].nunique()}",
        f"${gcp_df['cost_usd'].sum():,.2f}",
        f"${gcp_df.groupby('date')['cost_usd'].sum().mean():,.2f}",
        f"{(gcp_df['cost_usd'] < 0).sum()}",
        f"{gcp_key_duplicates}",
        f"{gcp_df.isnull().sum().sum()}"
    ]
})

print("\n" + "="*80)
print("COMPREHENSIVE DATA SUMMARY")
print("="*80 + "\n")
print(summary_stats.to_string(index=False))

## Conclusion

This profiling analysis has revealed:
1. Both datasets contain ~2,900 records covering approximately 100 days
2. Data structure is consistent between AWS and GCP with equivalent columns
3. **7 significant data quality risks** have been identified requiring remediation
4. Negative costs exist (likely credits/refunds) requiring special handling
5. Service naming is inconsistent across clouds (appears to be synthetic test data)

Next steps: Proceed to Part B (Data Modeling) to design a robust warehouse schema that addresses these quality concerns.